In [76]:
import jieba;
jieba.enable_paddle();

import copy;
import math;

#stage 1 read file
doc_list=[];
with open("./test.txt","rt",encoding='utf-8') as ifile:
    for line in ifile :
        lol=jieba.cut(line);
        doc_list.append(lol);
#test print
#for line in doc_list:
#    print('/'.join(line));
    

def scan_doc_list(inlist):
    docs={
        "total_doc_count":0,
        "total_word_count":0,
        "stat":dict(),# word -> {count,idf}
        "doc":list()#index -> doc
    };
    
    #scan every doc in lista
    for raw_doc in inlist:
        doc=scan_doc(raw_doc);
        docs["doc"].append(doc);
        #merge stat loop
        for word,info in doc["stat"].items():
            if word in docs["stat"]:
                docs["stat"][word]["count"]+=doc["stat"][word]["count"];
            else:
                docs["stat"][word]={};
                docs["stat"][word]["name"]=word;
                docs["stat"][word]["count"]=copy.deepcopy(doc["stat"][word]["count"]);
        #end of merge loop
        docs["total_doc_count"]+=1;
        docs["total_word_count"]+=doc["total_word_count"];
    #end of scan
    
    for word,info in docs["stat"].items():
        info["idf"]=math.log((docs["total_word_count"]*docs["total_doc_count"])/(info["count"]));
    #end of idf cal
    
    for doc in docs["doc"]:
        for word,info in doc["stat"].items():
            tr=(info["count"])/(doc["total_word_count"]);
            idf=docs["stat"][word]["idf"];
            info["tfidf"]=(tr)*(idf);
        #end of a word
    #end of a doc
    return docs;
#end of scan_doc_list

def scan_doc(inlist):
    doc={
        "total_word_count":0,
        "stat":{}# word -> {count,idf}
    };
    
    #scan thorogh list
    for word in inlist:
        if word in doc["stat"]:
            doc["stat"][word]["count"]+=1;
        else:
            doc["stat"][word]={};
            doc["stat"][word]["name"]=word;
            doc["stat"][word]["count"]=1;
        doc["total_word_count"]+=1;
    #end of loop
   
    return doc;
#end of scan_doc
result=scan_doc_list(doc_list)
print(result["stat"]);

Paddle enabled successfully......


{'this': {'name': 'this', 'count': 2, 'idf': 7.470224135899966}, ' ': {'name': ' ', 'count': 16, 'idf': 5.390782594220131}, 'is': {'name': 'is', 'count': 1, 'idf': 8.163371316459912}, 'line': {'name': 'line', 'count': 3, 'idf': 7.064759027791802}, '1': {'name': '1', 'count': 1, 'idf': 8.163371316459912}, '\n': {'name': '\n', 'count': 14, 'idf': 5.5243139868446525}, '2': {'name': '2', 'count': 1, 'idf': 8.163371316459912}, '4': {'name': '4', 'count': 2, 'idf': 7.470224135899966}, '為': {'name': '為', 'count': 4, 'idf': 6.7770769553400205}, '什麼': {'name': '什麼', 'count': 4, 'idf': 6.7770769553400205}, '聖結石': {'name': '聖結石', 'count': 1, 'idf': 8.163371316459912}, '會': {'name': '會', 'count': 2, 'idf': 7.470224135899966}, '被': {'name': '被', 'count': 3, 'idf': 7.064759027791802}, '酸': {'name': '酸', 'count': 1, 'idf': 8.163371316459912}, '而': {'name': '而', 'count': 1, 'idf': 8.163371316459912}, '這群': {'name': '這群', 'count': 2, 'idf': 7.470224135899966}, '人': {'name': '人', 'count': 4, 'idf': 6.77